<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/CodeBase/DLT_Project_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1
!pip install torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
%cd /content/drive/MyDrive/CSE8803_DLT_Project/

/content/drive/MyDrive/CSE8803_DLT_Project


In [26]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm
from ashish_test import utils
from utils import *
import json
import pandas as pd
import torch
# from torch.utils.data import Dataset, DataLoader
import time
import evaluate
from datasets import Dataset, load_dataset
import random

In [27]:
dir_path = '/content/drive/MyDrive/finqa_dataset'
os.chdir(dir_path)

In [113]:
with open(f"{dir_path}/train.json") as input_file:
        train_data = json.load(input_file)

with open(f"{dir_path}/dev.json") as input_file:
        valid_data = json.load(input_file)

with open(f"{dir_path}/test.json") as input_file:
        test_data = json.load(input_file)

In [114]:
assert len(train_data) == 6251, "The train data seems to be off, please check"
assert len(valid_data) == 883, "The validation data seems to be off, please check"
assert len(test_data) == 1147, "The test data seems to be off, please check"

Loading Ground Truth Serialized Dataset

In [76]:
df_serialized=pd.read_csv(f"{dir_path}/df_combined_table_ser_sample3000.csv")

In [77]:
df_serialized.columns

Index(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'id',
       'dataset', 'prompt', 'serialized_text_gpt35'],
      dtype='object')

In [78]:
df_serialized.shape

(3000, 9)

In [117]:
train_data=pd.DataFrame(train_data).merge(df_serialized[df_serialized['dataset']=='train'][['id','serialized_text_gpt35']],on='id',how='inner').to_dict(orient='records')

In [118]:
train_data[0].keys()

dict_keys(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'table_retrieved', 'text_retrieved', 'table_retrieved_all', 'text_retrieved_all', 'serialized_text_gpt35'])

In [119]:
train_data[0]['qa']

{'question': 'for acquired customer-related and network location intangibles , what is the expected annual amortization expenses , in millions?',
 'answer': '7.4',
 'explanation': '',
 'ann_table_rows': [],
 'ann_text_rows': [0, 1],
 'steps': [{'op': 'add1-1', 'arg1': '75.0', 'arg2': '72.7', 'res': '147.7'},
  {'op': 'divide1-2', 'arg1': '#0', 'arg2': '20', 'res': '7.4'}],
 'program': 'add(75.0, 72.7), divide(#0, 20)',
 'gold_inds': {'text_0': 'american tower corporation and subsidiaries notes to consolidated financial statements ( 3 ) consists of customer-related intangibles of approximately $ 75.0 million and network location intangibles of approximately $ 72.7 million .',
  'text_1': 'the customer-related intangibles and network location intangibles are being amortized on a straight-line basis over periods of up to 20 years .'},
 'exe_ans': 7.385,
 'tfidftopn': {'text_8': 'the customer-related intangibles and network location intangibles are being amortized on a straight-line basis 

In [120]:
def prepare_data(json_data, verbose=True):
    net=[]
    err_cnt=0
    for example in json_data:
        question = example["qa"]["question"]
        table = example["table"]
        id=example["id"]
        table_ori=example["table_ori"]
        answer_steps=example["qa"]["steps"]
        serialized_text=example["serialized_text_gpt35"]
        table_text = ""
        for row in table[1:]:
            this_sent = table_row_to_text(table[0], row)
            table_text += this_sent
        try:
            steps_text = format_steps(example["qa"]["steps"])
            inputs = {"id":id, "context": serialized_text, "question": question, "answer": steps_text,\
                      "table":table_ori,"answer_steps":answer_steps}
            net.append(inputs)
        except:
            err_cnt+=1
            if verbose:
                print ("-"*25)
                print (example["filename"])
                print (example["qa"]["steps"])
                print ("-"*25+"\n")
    if err_cnt>0:
        print ("Net Errors:",err_cnt)
    return net

In [121]:
## Preparing huggingface dataset
data_splits = {'train': train_data}
datasets = {split: Dataset.from_pandas(pd.DataFrame(prepare_data(data, False))) for split, data in data_splits.items()}

Net Errors: 98


In [122]:
index=0
datasets['train'][index]

{'id': 'AMT/2012/page_121.pdf-1',
 'context': 'The table represents the preliminary purchase price allocation for an acquisition. The acquisition resulted in the acquisition of current assets amounting to $8,763 and non-current assets amounting to $2,332. The non-current assets included property and equipment worth $26,711 and intangible assets worth $21,079. Other non-current liabilities amounting to ($1,349) were also acquired. The fair value of net assets acquired was $57,536, and the goodwill resulting from the acquisition was $5,998.',
 'question': 'for acquired customer-related and network location intangibles , what is the expected annual amortization expenses , in millions?',
 'answer': 'Step 1: Add 75.0 and 72.7. This gives the result: 147.7 ####### Step 2: Divide 147.7 by 20. This gives the result: 7.4',
 'table': [['', 'Preliminary Purchase Price Allocation'],
  ['Current assets', '$8,763'],
  ['Non-current assets', '2,332'],
  ['Property and equipment', '26,711'],
  ['Intan

In [123]:
step_pattern = r"Step \d+"
import re

In [124]:
def num_steps_dict_fn(example):
    example["num_steps"] = len(re.findall(step_pattern, example["answer"]))
    return example

In [125]:
datasets['train']=datasets['train'].map(num_steps_dict_fn)
# datasets['valid']=datasets['valid'].map(num_steps_dict_fn)
# datasets['test']=datasets['test'].map(num_steps_dict_fn)

Map:   0%|          | 0/2902 [00:00<?, ? examples/s]

In [126]:
datasets['train']

Dataset({
    features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
    num_rows: 2902
})

**Filtering Datasets for 1 Step**

In [127]:
datasets['train']=datasets['train'].filter(lambda example: example['num_steps']==1)
# datasets['valid']=datasets['valid'].filter(lambda example: example['num_steps']==1)
# datasets['test']=datasets['test'].filter(lambda example: example['num_steps']==1)

Filter:   0%|          | 0/2902 [00:00<?, ? examples/s]

In [128]:
datasets

{'train': Dataset({
     features: ['id', 'context', 'question', 'answer', 'table', 'answer_steps', 'num_steps'],
     num_rows: 1713
 })}

Loading Model

In [73]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [74]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token

Enter your Hugging Face token: ··········


In [75]:
# model_id =  "NousResearch/Llama-2-7b-hf"
model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",use_auth_token=hf_token)

tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [150]:
datasets['train'][0]['answer'],datasets['train'][36]['answer'],datasets['train'][21]['answer'],datasets['train'][5]['answer']

('Step 1: Divide 1733 by 2655. This gives the result: 65.3%',
 'Step 1: Add 130 and 294. This gives the result: 44.2%',
 'Step 1: Multiply 0.0425 by 1000. This gives the result: 42.5',
 'Step 1: Subtract 8 from 15. This gives the result: 7')

In [133]:
import warnings
warnings.filterwarnings('ignore')
# index = 10 (question not understood clearly), 11 (answer and context irrelevant to the question being asked)
index = 0

context_table=datasets['train'][index]['table']
context = datasets['train'][index]['context']
answer = datasets['train'][index]['answer']
question = datasets['train'][index]['question']

prompt = f"""
Answer the question in a single step by extracting relevant values from the context and performing necessary arithmetic operation.

### Context:
{context}

### Question:
{question}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=128,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the question in a single step by extracting relevant values from the context and performing necessary arithmetic operation.

### Context:
The table above provides a breakdown of reorganization items for the year 2013. The first item, labeled "Labor-related deemed claim (1)", had a value of $1,733. The second item, labeled "Aircraft and facility financing renegotiations and rejections (2), (3)", had a value of $325. The third item, labeled "Fair value of conversion discount (4)", had a value of $218. The fourth item, labeled "Professional fees", had a value of $199. The fifth item, labeled "Other", had a value of $180. The total value of all reorganization items for the year 2013 was $2,655.

### Question:
what is the percent of the labor-related deemed claim as part of the total reorganization items net in 2013

### Answer:

-----------------------------------------